# BG + FUS Attenuation Model (Interactive)

This notebook models how the basal ganglia (BG) select between two competing cortical action drives (A1 and A2), and how a *focused ultrasound (FUS)*-style modulation can transiently attenuate the dominant drive to observe action-flips or shifts in selection.

### Network architecture
- **Cortex [A1, A2]**: two competing input signals.
- **Basal Ganglia**: implements selection via disinhibition of one channel.
- **Thalamus**: relays the winner back to cortex/execution.

### Sliders & descriptions
- **A1**: Strength of cortical drive for channel 1 (initial dominant channel).
- **A2**: Strength of cortical drive for channel 2.
- **kappa**: Fractional attenuation applied to A1 during the FUS window (0 = no attenuation; 1 = full suppression of A1 in window).
- **fus_start**: Time in seconds when the FUS window starts.
- **fus_dur**: Duration in seconds of the FUS window.
- **T**: Total simulation time in seconds.
- **seed**: Random seed (integer) for reproducibility.

Use the sliders below to explore how parameter changes affect action selection and output behaviour.


In [ ]:
# Colab-friendly dependency install (safe to run locally as well)
try:
    import google.colab  # type: ignore
    %pip -q install nengo matplotlib ipywidgets
except Exception:
    pass

In [ ]:
import numpy as np
import nengo
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, interact

# Disable decoder cache (helps with hosted environments)
nengo.rc.set("decoder_cache", "enabled", "False")

np.random.seed(0)

In [ ]:
def run_sim(A1=0.8, A2=0.6, kappa=0.6, fus_start=0.5, fus_dur=0.3, T=1.2, seed=0):
    """Simulate BG + Thalamus action-selection with FUS attenuation on A1."""
    seed = int(seed)
    np.random.seed(seed)

    def fus_scale_fn(t):
        return (1.0 - kappa) if (fus_start <= t <= fus_start + fus_dur) else 1.0

    model = nengo.Network(label="BG + Thalamus with FUS attenuation")
    with model:
        cortex = nengo.Node(lambda t: [A1 * fus_scale_fn(t), A2], label="Cortex [A1,A2]")
        bg = nengo.networks.BasalGanglia(dimensions=2)
        th = nengo.networks.Thalamus(dimensions=2)

        nengo.Connection(cortex, bg.input, synapse=0.02)
        nengo.Connection(bg.output, th.input, synapse=0.02)

        p_ctx = nengo.Probe(cortex)
        p_bg  = nengo.Probe(bg.output, synapse=0.05)
        p_th  = nengo.Probe(th.output, synapse=0.05)

    with nengo.Simulator(model, seed=seed) as sim:
        sim.run(T)

    t = sim.trange()
    ctx = sim.data[p_ctx]
    bg_out = sim.data[p_bg]
    th_out = sim.data[p_th]

    plt.figure(figsize=(8,3))
    plt.plot(t, ctx[:,0], label="A1 (FUS-attenuated)")
    plt.plot(t, ctx[:,1], label="A2")
    plt.axvspan(fus_start, fus_start + fus_dur, alpha=0.15, label="FUS window")
    plt.legend(); plt.ylabel("Cortex"); plt.xlabel("Time (s)")
    plt.show()

    plt.figure(figsize=(8,3))
    plt.plot(t, bg_out)
    plt.ylabel("BG output"); plt.xlabel("Time (s)")
    plt.show()

    plt.figure(figsize=(8,3))
    plt.plot(t, th_out)
    plt.ylabel("Thalamus output"); plt.xlabel("Time (s)")
    plt.show()

    return dict(t=t, cortex=ctx, bg=bg_out, th=th_out)

In [ ]:
interact(
    run_sim,
    A1=FloatSlider(min=0.0, max=1.2, step=0.05, value=0.8),
    A2=FloatSlider(min=0.0, max=1.2, step=0.05, value=0.6),
    kappa=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.6),
    fus_start=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5),
    fus_dur=FloatSlider(min=0.05, max=0.7, step=0.05, value=0.3),
    T=FloatSlider(min=0.5, max=2.0, step=0.1, value=1.2),
    seed=IntSlider(min=0, max=10, step=1, value=0)
);

### Notes
- **Interpretation:** BG output reflects GPi/SNr inhibitory drive; lower BG output = stronger disinhibition of thalamus.
- **Look for:** When `kappa` is moderate, the weaker drive may win during the window. When `kappa` is high, you may see both thalamic channels elevate (loss of selectivity).
- **Next steps:** Add noise, oscillations, map to behavioural tasks (RT/SSRT), and derive EEG-like predictions.